In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop


from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import  plot_model
from sklearn.model_selection import cross_val_score, train_test_split

from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import scipy.io as sio
from scipy import signal
from scipy.fftpack import fft,fftshift
import random
import sys


In [ ]:
if tf.test.is_gpu_available():
    print('GPU acceleration is enabled')
else:
    print('GPU acceleration is not enabled')

### define model

In [ ]:
def MSSA_Block(input,conv_kernel_sizes,filter_size,dropout_rate):
    conv_attention_list=[]
    for kernel_size in conv_kernel_sizes:
        t_conv = keras.layers.Conv1D(filters=filter_size, kernel_size=kernel_size,padding="same")(input)
        t_conv = keras.layers.BatchNormalization()(t_conv)
        # t_conv=keras.layers.Activation('tanh')(t_conv)
        t_conv_attention=keras.layers.Attention()([t_conv,t_conv])
        conv_attention_list.append(t_conv_attention)
        pass

    concat=K.concatenate(conv_attention_list)
    concat=keras.layers.ReLU()(concat)
    concat=keras.layers.Dropout(dropout_rate)(concat)
    return concat


def model_multiconv1d_attention(input_shape,blcok_num,conv_kernel_sizes,filter_size,dropout_rate):
    input_layer = keras.layers.Input(shape=input_shape,name='input_layer')

    concat=input_layer
    for n in range(blcok_num):
        concat=MSSA_Block(input=concat,conv_kernel_sizes=conv_kernel_sizes,filter_size=filter_size,dropout_rate=dropout_rate)

    gap = keras.layers.GlobalAveragePooling1D()(concat)

    output_layer = keras.layers.Dense(300, activation="linear")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

### load data

In [ ]:
train_X_5fold=[]
train_Y_5fold=[]
valid_X_5fold=[]
valid_Y_5fold=[]

for n in range(5):
    train_valid_dataset=sio.loadmat('./data/train_valid_data_'+str(n)+'_fold.mat')

    train_X=train_valid_dataset['train_data_X']
    train_Y=train_valid_dataset['train_data_Y']

    train_X_5fold.append(train_X)
    train_Y_5fold.append(train_Y)

    valid_X=train_valid_dataset['valid_data_X']
    valid_Y=train_valid_dataset['valid_data_Y']

    valid_X_5fold.append(valid_X)
    valid_Y_5fold.append(valid_Y)



### weighted mse

In [ ]:
alpha=5
def weighted_mse(y_true, y_pred):
    mse_square = K.square(y_pred - y_true)

    condition = K.greater(y_true, 0)
    mse_square_adjusted = K.switch(condition, alpha * mse_square, mse_square)

    mse = K.mean(mse_square_adjusted)

    return mse


### train model 5 fold

In [ ]:
batch_size=32
epochs=550

adam=keras.optimizers.Adam(lr=0.001,decay=1e-6)

blcok_num=4
conv_kernel_sizes=[3,7,15]
filter_size=32
dropout_rate=0.2

for n in range(5):
    print('train no.'+str(n)+' fold model')

    train_X=train_X_5fold[n]
    train_Y=train_Y_5fold[n]

    valid_X=valid_X_5fold[n]
    valid_Y=valid_Y_5fold[n]


    train_X_input=train_X.reshape((train_X.shape[0],train_X.shape[1],1))
    valid_X_input=valid_X.reshape((valid_X.shape[0],valid_X.shape[1],1))

    # ............................................................................................
    model=model_multiconv1d_attention(train_X_input.shape[1:],blcok_num=blcok_num,conv_kernel_sizes=conv_kernel_sizes,filter_size=filter_size,dropout_rate=dropout_rate)
    model.summary()
    
    model.compile(
        optimizer=adam,
        # loss='mean_squared_error',
        loss=weighted_mse,
    )

    history = model.fit(
        train_X_input,
        train_Y,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(valid_X_input,valid_Y),
        verbose=1
    )

    # ............................................................................................
    modle_name="model_multiconv1d_attention"+'_'+str(n)+'_fold'
    model.save(modle_name+'.h5') 

    plt.figure()  
    plt.plot(history.history['loss'])
    plt.plot(history.history["val_loss"])
    plt.ylabel("loss", fontsize="large")
    plt.xlabel("epoch", fontsize="large")
    plt.legend(["train", "val"], loc="best")
    plt.savefig(modle_name+'_loss.png')
